In [1]:
from thefuzz import fuzz
from thefuzz import process
import pandas as pd
import names

In [2]:
def load_data(csv_path = './data') -> tuple[pd.DataFrame, pd.DataFrame, dict[str, pd.DataFrame]]:
    """
    Load transaction, customer and name data.
    
    Args:
        csv_path (str, optional): Path to the stored data. Defaults to './data'

    Returns:
        tuple: A tuple containing the transactions dataframe, the customers dataframe and the reference names dictionary.
    """

    # Reading data from csv
    transactions = pd.read_csv(f'{csv_path}/transactions.csv')
    customers = pd.read_csv(f'{csv_path}/customers.csv')

    # Getting most common names from the US Census (public data)
    # It is fetched through the module "names" and stored in the package data
    reference_names = {}
    for key, path in names.FILES.items():
        column_names = ['name', 'pct', 'sum_pct', 'position']

        df = pd.read_csv(
            path,
            sep=r'\s+',
            names=column_names, 
            nrows=1000,
        )
        reference_names[key] = df
    
    return transactions, customers, reference_names

In [3]:
def get_best_score(name: str, choices: list[str]) -> int:
    """
    Find the best score for a match between the name and the choices.

    Args:
        name (str): The name to be matched against the choices.
        choices (list[str]): A list with the possible choices to match the name to.

    Returns:
        int: The biggest score.
    """
    best, score = process.extractOne(name, choices, scorer=fuzz.token_set_ratio)
    return score

In [4]:
def extract_best(name: str, choices: list[str], threshold: int = 75) -> str | None:
    """
    Wrapper for the extractOne function which allows it to be called from apply.
    
    Args:
        name (str): The name to be matched against the choices.
        choices (list[str]): A list with the possible choices to match the name to.
        threshold (int, optional): The threshold to filter the results by (inclusive).

    Returns:
        Optional[str]: The best match, if there's one with score bigger than the threshold.
    """
    result = process.extractOne(name, choices, scorer=fuzz.token_set_ratio, score_cutoff=threshold)
    if result is not None:
        return result[0]
    return None

In [5]:
def select_best_full_name(row: pd.Series) -> str:
    """
    Find the most probable full name using the scores columns.

    Args:
        row (pd.Series): A pandas Series with the columns customer_name_1, customer_name_2, score_1, score_2, score_last_1, score_last_2.

    Returns:
        str: The most probable full name.
    """
    if row['score_1'] > row['score_2']:
        best_first = row['customer_name_1'].split(' ')[0]
    elif row['score_2'] > row['score_1']:
        best_first = row['customer_name_2'].split(' ')[0]
    else:
        best_first = row['customer_name_1'].split(' ')[0]

    if row['score_last_1'] > row['score_last_2']:
        best_last = row['customer_name_1'].split(' ')[1]
    elif row['score_last_2'] > row['score_last_1']:
        best_last = row['customer_name_2'].split(' ')[1]
    else:
        best_last = row['customer_name_1'].split(' ')[1]

    return f"{best_first} {best_last}"

In [8]:
if __name__ == '__main__':
    transactions, customers, reference_names = load_data()
    customers_names = customers['customer_name'].tolist()
    transactions['external_name'] = transactions['customer_name'].apply(lambda name: extract_best(name, customers_names))

    merged_df = pd.merge(
        transactions,
        customers,
        left_on='external_name',
        right_on='customer_name',
        how='left',
        suffixes=('_1', '_2')
    )

    merged_df = merged_df.fillna(' ')

    merged_df.drop(columns=['external_name'], inplace=True)
    merged_df['score_1'] = merged_df['customer_name_1'].apply(lambda name: get_best_score(name.split(' ')[0], reference_names['first:female']['name'].to_list()+reference_names['first:male']['name'].to_list()))
    merged_df['score_2'] = merged_df['customer_name_2'].apply(lambda name: get_best_score(name.split(' ')[0], reference_names['first:female']['name'].to_list()+reference_names['first:male']['name'].to_list()))

    merged_df['score_last_1'] = merged_df['customer_name_1'].apply(lambda name: get_best_score(name.split(' ')[1], reference_names['last']['name'].to_list()))
    merged_df['score_last_2'] = merged_df['customer_name_2'].apply(lambda name: get_best_score(name.split(' ')[1], reference_names['last']['name'].to_list()))

    merged_df['customer_name'] = merged_df.apply(lambda row: select_best_full_name(row), axis=1)
    merged_df.drop(columns=['customer_name_1', 'customer_name_2', 'score_1', 'score_2', 'score_last_1', 'score_last_2'], inplace=True)

    display(merged_df)

,transaction_id,amount,transaction_date,customer_id,email,customer_name
0,101,784,2023-01-31,1,john.smith@email.com,John Smith
1,102,659,2023-02-28,2,jane.doe@email.com,Jane Doe
2,103,729,2023-03-31,3,m.johnson@email.com,Michael Jonson
3,104,292,2023-04-30,4,c.lee@email.com,Chris Lee
4,105,935,2023-05-31,5,k.davis@email.com,Katherine Davis
5,106,863,2023-06-30,6,d.martinez@email.com,David Martinez
6,107,807,2023-07-31,7,s.brown@email.com,Sarah Brown
7,108,459,2023-08-31,8,j.wilson@email.com,James Wilson
8,109,109,2023-09-30,9,p.taylor@email.com,Patricia Taylor
9,110,823,2023-10-31,10,r.miller@email.com,Robert Miller
